### 서브그래프를 호출하는 함수 

함수는 노드의 상태 업데이트를 반환하기 전, 서브그래프 호출을 위해 입력 상태를 서브그래프 상태로 변환하고 서브 그래프 결과를 다시 상위 상태로 복원하는 과정을 수행해야 한다. 

In [1]:
from typing import TypedDict
from langgraph.graph import START, StateGraph

class State(TypedDict):
    foo: str # state key

# 서브 그래프를 직접 호출하지 않으므로 state key를 공유하지 않음 
class SubgraphState(TypedDict):
    bar: str 
    baz: str 
    
# 서브 그래프 정의 
def subgraph_node(state: SubgraphState):
    return {"bar": state['bar'] + "baz"}

subgraph_builder = StateGraph(SubgraphState)
subgraph_builder.add_node("subgraph_node", subgraph_node)
subgraph_builder.add_edge(START, "subgraph_node")
subgraph = subgraph_builder.compile() # 서브그래프 정의 


In [2]:
# 서브그래프를 호출하는 부모 그래프 정의 

def node(state: State):
    # 부모 그래프의 상태를 서브 그래프 상태로 변환 
    response = subgraph.invoke({'bar':state['foo']})
    # 응답을 다시 부모 그래프의 상태로 변환 
    return {'foo': response['bar']}

builder = StateGraph(State)
builder.add_node("node", node)
builder.add_edge(START, "node")
graph = builder.compile()

In [3]:
initial_state = {'foo':"hello"}
result = graph.invoke(initial_state)

print(
    f"Result: {result}"
)

Result: {'foo': 'hellobaz'}


### 멀티 에이전트 

애플리케이션을 여러 개의 소규모 독립 에이전트로 분할한 후, 이들을 멀티 에이전트 시스템으로 구성하는 방안을 강구할 수 있다. 독립 에이전트는 프롬프트와 LLM 호출만으로 간단하게 구성할 수 있고, ReAct Agent처럼 복잡하게 구현하는 것도 가능하다. 

- 독립 Agent가 여러 개의 Tool을 활용하는 방식

- Superviser 방식: 모든 에이전트가 슈퍼바이저 하나의 에이전트와 통신하며, 슈퍼바이저 에이전트는 이후 호출할 에이전트를 결정한다. 

- 네트워크 방식: 각 에이전트가 다른 에이전트와 상호 통신하여 모든 에이전트가 다음에 실행될 에이전트를 결정 가능하다.

- 계층형 방식: 여러 슈퍼바이저를 총괄하는 하나의 슈퍼바이저를 활용하여 관리하도록 설계된 멀티 에이전트 시스템을 정의한다.

In [32]:
from typing import Literal
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, MessagesState, START, END
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from pydantic import BaseModel 

openai_api_key = load_dotenv("../../../.env")

class SupervisorDecision(BaseModel):
    next: Literal['researcher','coder','FINISH']
    
# 모델 초기화 
model = ChatOpenAI(model = 'gpt-4o-mini', temperature=0)
model = model.with_structured_output(SupervisorDecision) # 모델의 출력 형식을 정의 

# 사용 가능한 에이전트 정의 
agents = ['researcher', 'coder']

system_prompt1 = f'''당신은 다음 서브에이전트 사이의 대화를 관리하는 슈퍼바이저입니다. 
서브에이전트: {agents}. 아래 사용자 요청에 따라 다음으로 행동할 서브에이전트를 지목하세요,
각 서브에이전트는 임무를 수행하고 결과와 상태를 응답합니다. 실행할 서브 에이전트가 없거나 작업이 완료되면 FINISH로 응답하세요'''

system_prompt2 = f'''위 대화를 바탕으로, 다음으로 행동할 서브에이전트는 누구인가요? 
아니면 FINISH 해야 합니까? 서브에이전트:{', '.join(agents)}, FINISH'''

def supervisor(state):
    messages = [
        SystemMessage(content=system_prompt1),
        *state['messages'],
        SystemMessage(content=system_prompt2)
    ]

    decision = model.invoke(messages)
    print(f"[Supervisor] Next agent: '{decision.next}'")  # 디버깅 로그

    return {"messages": state['messages'] + [AIMessage(content=f"Supervisor selected: {decision.next.strip()}")]}


In [ ]:
class AgentState(MessagesState):
    next: Literal['researcher', 'coder', 'FINISH']

# researcher 에이전트 정의 
def researcher(state):
    last_user_message = state['messages'][-1].content # 가장 최근 메세지의 content만 저장 
    response = f"Researcher is finding relevant information about: {last_user_message}"
    return {'messages': state['messages'] + [AIMessage(content=response)]}

# coder 에이전트 정의 
def coder(state):
    last_user_message = state['messages'][-1].content
    response = f"Coder is implementing code based on: {last_user_message}"
    return {'messages': state['messages'] + [AIMessage(content=response)]}

In [ ]:
workflow = StateGraph(AgentState)

# 노드 추가
workflow.add_node("supervisor", supervisor)
workflow.add_node("researcher", researcher)
workflow.add_node("coder", coder)

# supervisor 노드에서의 분기 처리
# 조건부 엣지 추가 
workflow.add_conditional_edges(
    "supervisor",
    lambda state: model.invoke([
        ('system', system_prompt1),
        *state['messages'],
        ('system', system_prompt2)
    ]).next,
    {
        "researcher": "researcher",
        "coder": "coder",
        "FINISH": END
    }
)

# researcher와 coder는 다시 supervisor로
workflow.add_edge("researcher", "supervisor")
workflow.add_edge("coder", "supervisor")

# 시작점 설정
workflow.set_entry_point("supervisor")

# 그래프 컴파일
graph = workflow.compile()

In [36]:
# 초기 메시지 상태 정의
inputs = {
    'messages': [('human', 'Create a Python web scraper that collects book titles from an online bookstore.')]
}

# 실행
result = graph.invoke(inputs)

# 출력 확인
result


[Supervisor] Next agent: 'coder'
[Supervisor] Next agent: 'FINISH'


{'messages': [HumanMessage(content='Create a Python web scraper that collects book titles from an online bookstore.', additional_kwargs={}, response_metadata={}, id='d148c1ae-6a7b-42f2-9d65-1ee29357a7ae'),
  AIMessage(content='Supervisor selected: coder', additional_kwargs={}, response_metadata={}, id='99550565-5942-403d-bc12-631437f0a85e'),
  AIMessage(content='💻 Coder is implementing code based on: Supervisor selected: coder', additional_kwargs={}, response_metadata={}, id='ac752b8f-a2a3-4ff2-8b03-8e4fa9837cf4'),
  AIMessage(content='Supervisor selected: FINISH', additional_kwargs={}, response_metadata={}, id='c41d7926-f7f5-4560-a9cd-11b1f8994265')]}